In [11]:
import os
import sys
print(os.getcwd())

C:\Users\dayeo\Documents\sentiment_analysis


In [12]:

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers # >= 4.x.x
!pip install torch

ERROR: Invalid requirement: '#'


In [13]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\dayeo\appdata\local\temp\pip-req-build-seefxxa_
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 8be1c06896c213279b46f0291839c554fe18b5f7


  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' 'C:\Users\dayeo\AppData\Local\Temp\pip-req-build-seefxxa_'


# Data Preparation and Training

In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [15]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [16]:
device = torch.device('cuda:0')

In [17]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [18]:
import pandas as pd
chatbot_data = pd.read_csv('C:/Users/dayeo/Downloads/sentiment_dialogues.csv')

In [19]:
chatbot_data.sample(n=10)

,Unnamed: 0,발화,감정
168937,168937,1시쯤.,중립
161861,161861,그럼. 그 답을 놓고서 문제 해결을 하면 되겠군요. 쉽진 않겠지만. 너무 복잡하게 ...,중립
31646,31646,예전에는 인간관계에 하나하나 상처받고 그랬는데 요새는 초연해진 것 같아. 예전에는 ...,행복
40397,40397,힘든 계약을 체결했더니 부장님이 격려해 주셔서 고마웠어.우리 부장님은 평상시에도 참...,행복
10515,10515,요새 마음이 불편하고 기분도 오락가락 알 수가 없어.짜증이 나다가 우울하다가 이상해...,분노
51856,51856,내가 할 일은 다 끝냈는데 가끔 엄마는 내가 아무것도 안한줄 알고 있어.너무 속상해...,불안
96754,96754,매일매일 열심히 운동하는데 왜 이렇게 피곤하지?그냥 혼자 이것저것 하는데 운동 방법...,슬픔
54478,54478,듣는 저도 황당하네요.어떻게 하면 좀 나아질까요?그것도 방법이겠네요.,당황
145544,145544,안구기증 같은걸 왜 해요?,중립
27995,27995,회사 업무에 많은 변화가 있었어.지난 시간동안 일해온것과 많이 달라서 혼란스러워.,불안


In [20]:
chatbot_data.loc[(chatbot_data['감정'] == "불안"), '감정'] = 0  #불안 => 0
chatbot_data.loc[(chatbot_data['감정'] == "당황"), '감정'] = 1  #당황 => 1
chatbot_data.loc[(chatbot_data['감정'] == "분노"), '감정'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['감정'] == "슬픔"), '감정'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['감정'] == "중립"), '감정'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['감정'] == "행복"), '감정'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['감정'] == "혐오"), '감정'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['발화'], chatbot_data['감정'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [21]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['지금 난 기분이 너무 좋아.오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.그냥 집에서 뒹굴뒹굴하고 넷플릭스 보고 놀 거야.', '5']
['집에만 가면 결혼하라는 부모님 때문에 너무 짜증나.응. 난 아직 결혼 생각도 없는데 말야.제일 싫은 건 자신의 세대에서 당연시 했던것 들을 우리에게도 강요하는 거야.', '1']
['대학교를 어디 갈 건지 자꾸 물어봐.선생님과 부모님이 자꾸 물어봐.귀찮아서 안 간다고 그랬어.', '2']
['남편이 요즘 건강이 안 좋아 보여.그동안 관계가 멀어져 제대로 된 식사도 안 챙겨줬거든. 이제 우리 나이가 대충 때우며 살 때가 아닌데 말이야. 아침에 힘들게 일어나는 걸 보면 내 탓인 것 같아.', '1']
['직장에서 상사가 자꾸 다른 동기만 예뻐해. 대놓고 그러니까 짜증 나고 동기도 괜히 미워져.일을 잘 하는 것도 아닌데 애교스러운 성격이라서 그런 것 같아.사실 그 동기라고 상사가 좋아서 살갑게 대하는 건 아니라고 생각해야겠어. ', '2']
['내 노후준비가 엉망이 됐어. 너무 억울해.노후준비를 계획했는데 남편의 사업실패로 물거품이 됐어.지금부터 천천히 다시 노후계획을 세워봐야겠어.', '3']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [22]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.13, random_state=0) #13% 데이터를 랜덤하게 테스트

In [23]:
print(len(dataset_train))
print(len(dataset_test))

190109
28408


In [24]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
#Setting parameters
max_len = 64 #텍스트 최대 길이
batch_size = 64 #가중치를 갱신하는 샘플의 개수
warmup_ratio = 0.1 #초반 warm up 비율
num_epochs = 15 #Full데이터 반복 횟수
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [26]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [27]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [28]:
# for data in train_dataloader:
#   print(data)
#   break

In [29]:
#kobert 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))[1]
        #bert model returns 'last_hidden_state' and 'pooler_output'

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [30]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device) #다 0.5로 해서 

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() #분류문제의 국룰

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
# train_dataloader

In [31]:
best_acc=0.0
best_loss=99999999
ckpt_path="C:/Users/dayeo/Downloads/" #your own path
ckpt_name=ckpt_path+"saved_model.pt"

In [32]:
#kobert 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        with torch.no_grad():
            out = model(token_ids, valid_length, segment_ids)
        test_loss=loss_fn(out,label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} test loss {}".format(e+1, test_acc / (batch_id+1),test_loss.data.cpu().numpy()))

    if test_acc>best_acc and test_loss.data.cpu().numpy()<best_loss:
      torch.save({'epoch':e+1,
                  'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'loss':test_loss.data.cpu().numpy()},
                 ckpt_name)
      best_loss=test_loss.data.cpu().numpy()
      bset_acc=test_acc
      
      print('current best model saved')
    

  0%|                                                                                         | 0/2971 [00:00<?, ?it/s]..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
  0%|                                                                                 | 1/2971 [00:00<44:20,  1.12it/s]

epoch 1 batch id 1 loss 2.0625057220458984 train acc 0.125


  1%|▍                                                                               | 18/2971 [00:07<21:51,  2.25it/s]


KeyboardInterrupt: 

# Inference

In [33]:
#새로운 문장 테스트
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0)
    
    model.eval()
    with torch.no_grad():

        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)

            valid_length= valid_length
            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)

            test_eval=[]
            
            values,indices=torch.topk(out,3)
            indices=indices.detach().cpu().numpy()
            for i in range(3):
                if indices.item(i)== 0:
                    test_eval.insert(i,"불안이")
                if indices.item(i)==1:
                    test_eval.insert(i,"당황이")
                if indices.item(i)==2:
                    test_eval.insert(i,"분노가")
                if indices.item(i)==3:
                    test_eval.insert(i,"슬픔이")
                if indices.item(i)==4:
                    test_eval.insert(i,"중립이")
                if indices.item(i)==5:
                    test_eval.insert(i,"행복이")
                if indices.item(i)==6:
                    test_eval.insert(i,"혐오가")
            for j in out:
                logits=j
                logits = logits.detach().cpu().numpy()
                
            print(">> 입력하신 내용에서 " + test_eval[0] +","+test_eval[1]+","+test_eval[2] +" 느껴집니다.")
            print(logits)

using cached model


In [34]:
checkpoint=torch.load(ckpt_name)

In [35]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 이쁜 너 참 밉기도 하다.
>> 입력하신 내용에서 행복이,중립이,슬픔이 느껴집니다.
[-2.575842   -1.9993618  -1.1298313   0.82227236  2.278468    4.1164427
 -1.0164168 ]


하고싶은 말을 입력해주세요 : 내가 널 어떻게 미워하겠어
>> 입력하신 내용에서 슬픔이,분노가,당황이 느껴집니다.
[-0.79527694 -0.54745007  0.27246687  4.675166   -1.1524129  -1.1596696
 -1.1079328 ]


